Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [34]:
import unicodecsv

def read_csv(fileName):
    with open('../res/' + fileName, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

In [35]:
###############################################################
enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily_engagement.csv')
project_submissions = read_csv('project_submissions.csv')
###############################################################

## Fixing Data Types

In [36]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date is None or isinstance(date, dt):
        return
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i is None or isinstance(i, int):
        return
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
# isinstance(enrollments[5]['cancel_date'], dt)
enrollments[0]

OrderedDict([('account_key', '448'),
             ('status', 'canceled'),
             ('join_date', datetime.datetime(2014, 11, 10, 0, 0)),
             ('cancel_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('days_to_cancel', 65),
             ('is_udacity', True),
             ('is_canceled', True)])

In [37]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

OrderedDict([('acct', '0'),
             ('utc_date', datetime.datetime(2015, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0)])

In [38]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

OrderedDict([('creation_date', datetime.datetime(2015, 1, 14, 0, 0)),
             ('completion_date', datetime.datetime(2015, 1, 16, 0, 0)),
             ('assigned_rating', 'UNGRADED'),
             ('account_key', '256'),
             ('lesson_key', '3176718735'),
             ('processing_state', 'EVALUATED')])

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [39]:
#####################################
total_rows_enrollment = len(enrollments)
total_rows_daily_engagement = len(daily_engagement)
total_rows_project_submissions = len(project_submissions)
enrollments_unique = {*map(lambda e: e['account_key'], enrollments)}
daily_engagement_unique = {*map(lambda e: e['acct'], daily_engagement)}
project_submissions_unique = {*map(lambda e: e['account_key'], project_submissions)}
us1 = len(enrollments_unique)
us2 = len(daily_engagement_unique)
us3 = len(project_submissions_unique)
# print(total_rows_enrollment, total_rows_daily_engagement, total_rows_project_submissions)
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

## Problems in the Data

In [40]:
#####################################
def update(each):
    each['account_key'] = each['acct']
    del(each['acct'])
    return each
#####################################
daily_engagement = [*map(lambda d: update(d), daily_engagement)]
## Rename the "acct" column in the daily_engagement table to "account_key".

In [41]:
daily_engagement[0]['account_key']

'0'

## Removing the test accounts

In [42]:

k = [*filter(lambda e: e not in daily_engagement_unique, enrollments_unique)]
# print(k)
udacity_test_accounts = {*map(lambda e: e['account_key'], [*filter(lambda d: d['is_udacity'], enrollments)])} 
# ###############
non_udacity_enrollments = [*filter(lambda e: e['account_key'] not in udacity_test_accounts, enrollments)]
non_udacity_daily_engagement = [*filter(lambda e: e['account_key'] not in udacity_test_accounts, daily_engagement)]
non_udacity_project_submissions = [*filter(lambda e: e['account_key'] not in udacity_test_accounts, project_submissions)]

## All paid students

In [43]:
# paid_students = {each['account_key']: each['join_date'] for each in non_udacity_enrollments if each['days_to_cancel'] == None or each['days_to_cancel'] > 7}
paid_students = {}
for each in non_udacity_enrollments:
    if each['days_to_cancel'] == None or each['days_to_cancel'] > 7:
        if each['account_key'] not in paid_students or each['join_date'] > paid_students[each['account_key']]:
            paid_students[each['account_key']] = each['join_date']
# print(paid_students)
# non_udacity_daily_engagement[0]

In [44]:
paid_engagement_in_first_week = []
for each in non_udacity_daily_engagement:
    if each['account_key'] in paid_students and 0 <= (each['utc_date'] - paid_students[each['account_key']]).days < 7:
#         if (each['utc_date'] - paid_students[each['account_key']]).days < 0:
#             print('OOPS!!!')
        paid_engagement_in_first_week.append(each)
print(len(paid_engagement_in_first_week))

paid_engagement_in_first_week[0]

6919


OrderedDict([('utc_date', datetime.datetime(2015, 1, 9, 0, 0)),
             ('num_courses_visited', 1),
             ('total_minutes_visited', 11.6793745),
             ('lessons_completed', 0),
             ('projects_completed', 0),
             ('account_key', '0')])

In [45]:
for each in paid_engagement_in_first_week:
    if each['num_courses_visited'] > 0:
        each['has_visited'] = 1
    else:
        each['has_visited'] = 0

In [53]:
from collections import defaultdict
from functools import reduce
def calculate_metric_of_one_week(dataFrame, metric=None, transform=False, transVar=None, agf=sum):
    data = defaultdict(list)
    if not transform:
        [*map(lambda d: data[d['account_key']].append(d[metric]), dataFrame)]
    else:
        [*map(lambda d: data[d['account_key']].append(transVar[0]) if d[metric] else data[d['account_key']].append(transVar[1]), dataFrame)] 
    return [*map(lambda e: agf(data[e]), data)]
# def calculate_metric_of_one_week(dataFrame, metric=None, agf=sum):
#     data = defaultdict(list)
#     [*map(lambda d: data[d['account_key']].append(d[metric]) if d[metric] > 0 else None, dataFrame)]
#     return [*map(lambda e: agf(data[e]), data)]
# #     return res
def get_stats(metric_entries):
    import numpy as np
    return {
        "mean": np.mean(metric_entries), 
        "stdv": np.std(metric_entries), 
        "min": min(metric_entries), 
        "max": max(metric_entries)
    }
############ time spent by students in first week ##########
time_spent_by_all_in_one_week_grouped_by_student = calculate_metric_of_one_week(paid_engagement_in_first_week, metric='total_minutes_visited')
print(get_stats(time_spent_by_all_in_one_week_grouped_by_student))
############ Lessons completed by students in first week ######################
lessons_completed_by_all_in_one_week_grouped_by_student = calculate_metric_of_one_week(paid_engagement_in_first_week, metric='lessons_completed')
print(get_stats(lessons_completed_by_all_in_one_week_grouped_by_student))
# ############ Indivisual days students visited atleast one nanodegree within first week ###############
# # paid_engagement_in_first_week[0]
days_spent_by_all_in_one_week_grouped_by_student = calculate_metric_of_one_week(paid_engagement_in_first_week, metric='num_courses_visited', transform=True, transVar=(1, 0))
print(get_stats(days_spent_by_all_in_one_week_grouped_by_student))

{'mean': 306.70832675342825, 'stdv': 412.99693340852957, 'min': 0.0, 'max': 3564.7332644989997}
{'mean': 1.636180904522613, 'stdv': 3.0025612998294231, 'min': 0, 'max': 36}
{'mean': 2.8673366834170855, 'stdv': 2.2551980029196814, 'min': 0, 'max': 7}


In [57]:
# non_udacity_project_submissions[0]
paid_engagement_in_first_week[0]
aid_engagement_first_week = {each['account_key'] for each in paid_engagement_in_first_week}

# aid_engagement_first_week
# for each in non_udacity_project_submissions:
#     if each['assigned_rating'] == 'PASSED':
        

{'0',
 '1',
 '10',
 '100',
 '1000',
 '1001',
 '1002',
 '1007',
 '1009',
 '101',
 '1011',
 '1013',
 '1014',
 '1015',
 '1019',
 '102',
 '1022',
 '1028',
 '1029',
 '103',
 '1030',
 '1031',
 '1032',
 '1033',
 '1036',
 '1038',
 '104',
 '1040',
 '1042',
 '1048',
 '1049',
 '105',
 '1050',
 '1052',
 '1053',
 '1054',
 '1055',
 '1056',
 '1058',
 '106',
 '1060',
 '1062',
 '1066',
 '1067',
 '107',
 '1070',
 '1071',
 '1072',
 '1073',
 '1074',
 '1075',
 '1077',
 '108',
 '1080',
 '1081',
 '1082',
 '1087',
 '1088',
 '1089',
 '109',
 '1090',
 '1094',
 '1095',
 '1096',
 '1098',
 '1099',
 '11',
 '110',
 '1102',
 '1103',
 '1107',
 '111',
 '1110',
 '1111',
 '1113',
 '1115',
 '1117',
 '1118',
 '112',
 '1121',
 '1123',
 '1124',
 '1128',
 '113',
 '1130',
 '1131',
 '1136',
 '1139',
 '114',
 '1140',
 '1142',
 '1143',
 '1144',
 '1146',
 '1147',
 '1149',
 '115',
 '1150',
 '1151',
 '1153',
 '1154',
 '1157',
 '116',
 '1160',
 '1161',
 '1163',
 '1167',
 '117',
 '1170',
 '1173',
 '1174',
 '1175',
 '1176',
 '1177',
 '

## Missing Engagement Records

## Checking for More Problem Records

In [50]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.

## Tracking Down the Remaining Problems

In [51]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [52]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [53]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

SyntaxError: invalid syntax (<ipython-input-53-40102216c630>, line 6)

## Refining the Question

In [ ]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students =

## Getting Data from First Week

In [ ]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

In [ ]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week = 

## Exploring Student Engagement

In [ ]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [ ]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [ ]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = total_minutes_by_account.values()
print 'Mean:', np.mean(total_minutes)
print 'Standard deviation:', np.std(total_minutes)
print 'Minimum:', np.min(total_minutes)
print 'Maximum:', np.max(total_minutes)

## Debugging Data Analysis Code

In [ ]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

## Lessons Completed in First Week

In [ ]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

## Number of Visits in First Week

In [ ]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.

## Splitting out Passing Students

In [ ]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

passing_engagement =
non_passing_engagement =

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.